Full Training and testing Done

In [1]:
%load_ext autoreload
%autoreload 2

# Environment prep for Colab
 - download git repo 
 - install required pip packages

In [2]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/
!pip -q install -r /content/Generative_Models/requirements.txt
%cd /content/

Cloning into 'Generative_Models'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 16171 (delta 0), reused 3 (delta 0), pack-reused 16162
Receiving objects: 100% (16171/16171), 1.00 GiB | 36.42 MiB/s, done.
Resolving deltas: 100% (11012/11012), done.
Checking out files: 100% (1688/1688), done.
/content/Generative_Models
/content


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/
!pip -q install -r /content/Generative_Models/requirements.txt
!pip install colorlog==4.0.2
%cd /content/

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Results’: File exists
/content/Generative_Models
  Found existing installation: colorlog 4.6.2
    Uninstalling colorlog-4.6.2:
      Successfully uninstalled colorlog-4.6.2
/content


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Connect Drive to save and restore results

In [5]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/Results
MAIN_SAVE_DIR = '/content/drive/My Drive/Results'
IMG_DIR = '/content/Generative_Models/data/.pokemon' #change .qbert here as well

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/Results’: File exists


# Settings

In [6]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
/


In [7]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [8]:
#os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
os.chdir('/content/Generative_Models')
print(os.getcwd())

/content/Generative_Models


# only on colab

In [9]:
!cat data/.pokemon/pokemon.tar.part* > data/.pokemon/pokemon.tar

In [10]:
!tar -xvf data/.pokemon/pokemon.tar --directory data/.pokemon/

Pokemon_LMDB/_training/
Pokemon_LMDB/_training/lock.mdb
Pokemon_LMDB/_training/data.mdb
Pokemon_LMDB/_validation/
Pokemon_LMDB/_validation/lock.mdb
Pokemon_LMDB/_validation/data.mdb
Pokemon_LMDB/meta_info.json


# Dataset loading

In [11]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

In [12]:
dataset_name='pokemon'
inputs_shape= image_shape=(100, 100, 3)
batch_size = 10
latents_dim = 100
intermediate_dim = 50
lmdb_dir = 'data/.pokemon/Pokemon_LMDB'

In [13]:
from transformation.lmdb_transformer import LmdbTransformer

In [14]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None
)

/content/Generative_Models/transformation/file_image_generator.py:51: UserWarning: No files found
  warnings.warn(msg)


Initializing Iterator training Number of images 591
Initializing Iterator validation Number of images 218


In [15]:
import numpy as np
_instance_scale=1.0
for data in val_generator:
    print(np.max(data['images']))
    break
    #print(data.keys())

1.0


In [16]:
data['images'][0].shape

TensorShape([400, 400, 3])

In [17]:
data.keys()

dict_keys(['images', 'label'])

# Model's Layers definition

In [18]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    flat_outputs_shape = np.prod(inputs_shape)
flat_outputs_shape

30000

In [19]:
menc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

venc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=flat_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [20]:
model_name = dataset_name+'VAE_Dense_reconst_1ell_1sharpdiff'
experiments_dir='/content/drive/My Drive/Results'+sep_local+model_name
print(experiments_dir)

/content/drive/My Drive/Results/pokemonVAE_Dense_reconst_1ell_1sharpdiff


In [21]:
from training.autoencoding_basic.autoencoders.VAE import VAE as AE

In [22]:
#inputs_shape=image_size

In [23]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    },
    
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    },
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [24]:
from utils.data_and_files.file_utils import create_if_not_exist

In [25]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [26]:
create_if_not_exist(_restore)
_restore

'/content/drive/My Drive/Results/pokemonVAE_Dense_reconst_1ell_1sharpdiff/var_save_dir'

In [27]:
#to restore trained model, set filepath=_restore

In [28]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=_restore
    )

  DEBUG    | Restore old models ...
  DEBUG    | Restore old models ...


  WARNING  | None
  WARNING  | None


Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (Input [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 100, 100, 25)      100       
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 25)      650       
_________________________________________________________________
flatten (Flatten)            (None, 250000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               25000100  
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dropout (Dropout)            (None, 100)            

  WARNING  | None
  WARNING  | None


Model: "inference_logvariance"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_logvariance_inputs [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense_3 (Dense)              (None, 100, 100, 25)      100       
_________________________________________________________________
dense_4 (Dense)              (None, 100, 100, 25)      650       
_________________________________________________________________
flatten_1 (Flatten)          (None, 250000)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               25000100  
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)     

  WARNING  | None
  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_8 (Dense)              (None, 30000)             1530000   
_________________________________________________________________
reshape (Reshape)            (None, 100, 100, 3)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 100, 100, 3)       12        
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 100, 3)       0

In [29]:
from evaluation.quantitive_metrics.sharp_difference import prepare_sharpdiff
from statistical.losses_utilities import similarity_to_distance
from statistical.ae_losses import expected_loglikelihood_with_lower_bound as ellwlb

In [30]:
ae.compile(loss={'x_logits': lambda x_true, x_logits: ellwlb(x_true, x_logits)+similarity_to_distance(prepare_sharpdiff([ae.batch_size]+ae.get_inputs_shape()))(x_true, x_logits)})

Model: "pokemonVAE_Dense_reconst_1ell_1sharpdiff"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (I [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
inference_logvariance (Function (None, 100)          25001250    inference_logvariance_inputs[0][0
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 100)]        0           inference_logvariance[0][0]      
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 100)]        0           tf_op_layer_Mul[0][0]            
___________________________________________________________

# Callbacks

In [31]:

from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [32]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [33]:
ms = ModelSaver(filepath=_restore)

In [34]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'/content/drive/My Drive/Results/pokemonVAE_Dense_reconst_1ell_1sharpdiff/csv_dir/pokemonVAE_Dense_reconst_1ell_1sharpdiff.csv'

In [35]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [36]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

# Model Training

In [ ]:
ae.fit(
    x=train_generator,
    input_kw='images',
    steps_per_epoch=int(5e3),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=train_generator,
    validation_steps=int(1e4)
)

Epoch 1/1000000

Epoch 00001: loss improved from inf to 7486.41797, saving model to /content/drive/My Drive/Results/pokemonVAE_Dense_reconst_1ell_1sharpdiff/var_save_dir
5000/5000 - 205s - loss: 7486.4180 - x_logits_loss: 7485.0615 - x_logpdf_loss: -1.3274e+00 - z_latents_loss: 2.4911 - val_loss: 3553.5220 - val_x_logits_loss: 3552.0803 - val_x_logpdf_loss: -1.2982e+00 - val_z_latents_loss: 2.4823
Epoch 2/1000000

Epoch 00002: loss improved from 7486.41797 to 6418.76221, saving model to /content/drive/My Drive/Results/pokemonVAE_Dense_reconst_1ell_1sharpdiff/var_save_dir
5000/5000 - 205s - loss: 6418.7622 - x_logits_loss: 6417.1665 - x_logpdf_loss: -1.2984e+00 - z_latents_loss: 2.5460 - val_loss: 3130.5796 - val_x_logits_loss: 3128.9463 - val_x_logpdf_loss: -1.3258e+00 - val_z_latents_loss: 2.5893
Epoch 3/1000000

Epoch 00003: loss improved from 6418.76221 to 6278.40234, saving model to /content/drive/My Drive/Results/pokemonVAE_Dense_reconst_1ell_1sharpdiff/var_save_dir
5000/5000 - 20

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=300)
print(f'inception_score mean: {is_mean.compute()}, sigma: {is_sigma.compute()}')

87916544/87910968 [==============================] - 1s 0us/step
calculating the inception_score mean ...


300it [2:13:32, 26.58s/it]

calculating the inception_score sigma ...


/usr/local/lib/python3.6/dist-packages/dask/array/routines.py:780: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
300it [2:15:29, 27.24s/it]

inception_score mean: [3.22672002e+81], sigma: 5.252197473666361e+163



## Frechet_inception_distance




In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, train_generator.as_numpy_iterator(), 
                                       tolerance_threshold=1e-6, 
                                       max_iteration=300, 
                                       batch_size=32)


In [ ]:
print(f'frechet inception distance: {fis_score.compute()}')

## perceptual_path_length_score

In [37]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [38]:
ppl_mean_score = perceptual_path_length_score(ae, train_generator.as_numpy_iterator(), 
                                              tolerance_threshold=1e-6, 
                                              max_iteration=300, 
                                              batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

58892288/58889256 [==============================] - 0s 0us/step


InvalidArgumentError: ignored

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae,  train_generator.as_numpy_iterator(), 
                                   tolerance_threshold=1e-6, 
                                   max_iteration=300)
print(f'precision score: {_precision_score}')

/content/Generative_Models/evaluation/generativity_metrics/precision_recall.py:84: RuntimeWarning: invalid value encountered in true_divide
  realism_score[begin1:end1] = self.D[nearest_indices[begin1:end1], 0] / np.min(distance_batch[0:end1-begin1, :], axis=1)
0it [00:00, ?it/s]/content/Generative_Models/evaluation/generativity_metrics/precision_recall.py:84: RuntimeWarning: invalid value encountered in true_divide
  realism_score[begin1:end1] = self.D[nearest_indices[begin1:end1], 0] / np.min(distance_batch[0:end1-begin1, :], axis=1)
                  

Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00290537s
Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00180173s
precision score: [0.]


## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, train_generator.as_numpy_iterator(), tolerance_threshold=1e-6, max_iteration=300)
print(f'recall score: {_recall_score}')

/content/Generative_Models/evaluation/generativity_metrics/precision_recall.py:84: RuntimeWarning: invalid value encountered in true_divide
  realism_score[begin1:end1] = self.D[nearest_indices[begin1:end1], 0] / np.min(distance_batch[0:end1-begin1, :], axis=1)
0it [00:00, ?it/s]/content/Generative_Models/evaluation/generativity_metrics/precision_recall.py:84: RuntimeWarning: invalid value encountered in true_divide
  realism_score[begin1:end1] = self.D[nearest_indices[begin1:end1], 0] / np.min(distance_batch[0:end1-begin1, :], axis=1)
                  

Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00241899s
Evaluating k-NN precision and recall with 10 samples...
Done evaluation in: 0.00196218s
recall score: [1.]


# Image Generation

## image reconstruction

### Training dataset

In [ ]:
from training.generators.from_images.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

10it [00:00, 74.26it/s]


In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:00, 40.09it/s]


## with Randomness

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, train_generator.as_numpy_iterator(), save_dir)

10it [00:00, 43.60it/s]


In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

10it [00:00, 44.74it/s]


### Complete Randomness

In [ ]:
from training.generators.from_images.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

32it [00:00, 103.54it/s]


In [ ]:
from training.generators.from_images.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, val_generator.as_numpy_iterator(), save_dir)

3it [00:00, 23.32it/s]

10it [00:00, 23.66it/s]
3it [00:00, 22.65it/s]

10it [00:00, 23.51it/s]
3it [00:00, 23.14it/s]

10it [00:00, 26.23it/s]
3it [00:00, 28.03it/s]

10it [00:00, 26.86it/s]
100%|██████████| 4/4 [00:01<00:00,  2.36it/s]
10it [00:00, 26.19it/s]
